IMPORT LIBRARY

In [12]:
import requests, re, json, lxml
from bs4 import BeautifulSoup

PARAMETER PENCARIAN

In [13]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36",
  }

params = {
    "q": "daging ayam",              # search query
    "tbm": "isch",                # image results
    "hl": "id",                   # language of the search
    "gl": "id",                   # country where search comes fro
  }

SEARCH 

In [14]:
html = requests.get("https://google.com/search", params=params, headers=headers, timeout=30)
soup = BeautifulSoup(html.text, 'html.parser')
google_images = []
all_script_tags = soup.select("script")

REGEX (tinggal copas copas dari internet doang asli wkwk)

In [15]:
# https://regex101.com/r/48UZhY/4
matched_images_data = "".join(re.findall(r"AF_initDataCallback\(([^<]+)\);", str(all_script_tags)))

matched_images_data_fix = json.dumps(matched_images_data)
matched_images_data_json = json.loads(matched_images_data_fix)
  
# https://regex101.com/r/VPz7f2/1
matched_google_image_data = re.findall(r'\"b-GRID_STATE0\"(.*)sideChannel:\s?{}}', matched_images_data_json)

# https://regex101.com/r/Jt5BJW/1
matched_google_images_thumbnails = ", ".join(
    re.findall(r'\[\"(https\:\/\/encrypted-tbn0\.gstatic\.com\/images\?.*?)\",\d+,\d+\]',
                   str(matched_google_image_data))).split(", ")

thumbnails = [bytes(bytes(thumbnail, "ascii").decode("unicode-escape"), "ascii").decode("unicode-escape") for thumbnail in matched_google_images_thumbnails]

# removing previously matched thumbnails for easier full resolution image matches.
removed_matched_google_images_thumbnails = re.sub(
        r'\[\"(https\:\/\/encrypted-tbn0\.gstatic\.com\/images\?.*?)\",\d+,\d+\]', "", str(matched_google_image_data))

# https://regex101.com/r/fXjfb1/4
# https://stackoverflow.com/a/19821774/15164646
matched_google_full_resolution_images = re.findall(r"(?:'|,),\[\"(https:|http.*?)\",\d+,\d+\]", removed_matched_google_images_thumbnails)

full_res_images = [
        bytes(bytes(img, "ascii").decode("unicode-escape"), "ascii").decode("unicode-escape") for img in matched_google_full_resolution_images
]

METHOD DOWNLOAD

In [49]:
import requests
from PIL import Image
import io
import time
       
def download(image,folder,iter):
    # Set the image URL
    url = image

    # Make a GET request to the URL and save the response
    try:
        response = requests.get(url)
    except :
        time.sleep(30)
        try:
           response = requests.get(url)
        except:
            time.sleep(30)
            pass
        

    try:
        images = Image.open(io.BytesIO(response.content))
        
        # Save the image to a file
        images.save("{}/tes_{}.jpg".format(folder,iter))
    except:
        pass

MASUKIN LINK KE ARRAY

In [18]:
for index, (metadata, thumbnail, original) in enumerate(zip(soup.select('.isv-r.PNCib.MSM1fd.BUooTd'), thumbnails, full_res_images), start=1):
    google_images.append(original)

In [30]:
len(google_images)

100

DOWNLOAD

In [50]:
iter = 0
Folder = 'tes'
for i in google_images:
    download(i,Folder,iter)
    iter = iter + 1

Before the sleep statement
After the sleep statement
